In [51]:

import cv2
import os
import glob
import time
import numpy as np

# Path to the images
image_path = "dataset/train/images/clip_one/*"

# Get all image files in the directory
image_files = sorted(glob.glob(image_path))

if not image_files:
    print("No images found in the specified directory.")
else:
    # Create a window
    cv2.namedWindow("Video", cv2.WINDOW_NORMAL)
    
    backSub = cv2.createBackgroundSubtractorMOG2(history=100)
    backSub.setBackgroundRatio(0.9)
    backSub.setDetectShadows(False)
    
    # Display each image in sequence
    for img_file in image_files:
        # Read the image
        frame = cv2.imread(img_file)
        # Convert to grayscale
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # Apply a binary threshold to get rid of the background
        _, frame = cv2.threshold(gray_frame, 125, 255, cv2.THRESH_TOZERO)
        frame = backSub.apply(frame)
        
        # Define a kernel for morphological operations
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))  # Use a rectangular kernel to filter out lines
        
        # Apply morphological operations to filter out lines
        frame = cv2.morphologyEx(frame, cv2.MORPH_OPEN, kernel)  # Remove small objects
        frame = cv2.morphologyEx(frame, cv2.MORPH_CLOSE, kernel)  # Close gaps in the objects
        
        # Use a line filter to remove linear structures
        lines = cv2.HoughLinesP(frame, 1, np.pi / 180, threshold=10, minLineLength=30, maxLineGap=30)
        if lines is not None:
            for line in lines:
                x1, y1, x2, y2 = line[0]
                cv2.line(frame, (x1, y1), (x2, y2), (0, 0, 0), 2)  # Draw black lines over detected lines to remove them
        
        # Apply Gaussian blur to the frame
        # frame = cv2.GaussianBlur(frame, (5, 5), 2)
        # _, frame = cv2.threshold(frame, 125, 255, cv2.THRESH_TOZERO)
        
        
        
        if frame is None:
            print(f"Could not read image: {img_file}")
            continue
        
        # Display the image
        cv2.imshow("Video", frame)
        
        # Wait for a short time to create video effect (30 FPS)
        if cv2.waitKey(33) & 0xFF == ord('q'):  # Press 'q' to exit
            break
        
    # Close all windows
    cv2.destroyAllWindows()

In [7]:
import cv2
import os
import glob
import numpy as np

# Path to the images
image_path = "dataset/train/images/clip_one/*"

# Get all image files in the directory
image_files = sorted(glob.glob(image_path))

if not image_files:
    print("No images found in the specified directory.")
else:
    # Create a window
    cv2.namedWindow("Video", cv2.WINDOW_NORMAL)

    backSub = cv2.createBackgroundSubtractorMOG2(history=0, varThreshold=1000)
    backSub.setBackgroundRatio(0.9)

    # Optical Flow parameters
    prev_gray = None
    thresh = 5  # Set your optical flow magnitude threshold

    for img_file in image_files:
        # Read the image
        frame = cv2.imread(img_file)
        if frame is None:
            print(f"Could not read image: {img_file}")
            continue

        # Convert to grayscale
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Apply a binary threshold to get rid of the background
        _, binary_frame = cv2.threshold(gray_frame, 150, 255, cv2.THRESH_TOZERO)
        fg_mask = backSub.apply(binary_frame)

        # Apply morphological operations to clean up noise
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
        fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_DILATE, kernel)

        # Apply Gaussian blur
        fg_mask = cv2.GaussianBlur(fg_mask, (5, 5), 2)

        # Compute optical flow if there is a previous frame
        if prev_gray is not None:
            flow = cv2.calcOpticalFlowFarneback(prev_gray, gray_frame, None, 0.5, 3, 5, 3, 5, 1.2, 0)
            
            # Compute magnitude and angle
            mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])

            # Create a mask for motion that exceeds the threshold
            motion_mask = (mag > thresh).astype(np.uint8) * 255

            # Apply mask to the frame
            filtered_frame = cv2.bitwise_and(frame, frame, mask=motion_mask)

            # Display the filtered frame
            cv2.imshow("Video", filtered_frame)
            cv2.imshow("fg_mask", fg_mask)
            cv2.imshow("gray", gray_frame)

        prev_gray = gray_frame  # Update previous frame for optical flow calculation

        # Wait for a short time to create a video effect (30 FPS)
        if cv2.waitKey(33) & 0xFF == ord('q'):  # Press 'q' to exit
            break

    # Close all windows
    cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np

# Load the video
# video_path = "volleyball_rolling.webm"
video_path = "volleyball_video.mp4"
cap = cv2.VideoCapture(video_path)

# Check if video loaded successfully
if not cap.isOpened():
    raise ValueError("Error opening video file.")

# Parameters for ball tracking
ball_radius_range = (3, 7)  # Considering the ball is around 5 pixels in size
# ball_radius_range = (25, 50)  # Considering the ball is around 5 pixels in size
bg_subtractor = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=30, detectShadows=True)

# Kalman Filter initialization
kalman = cv2.KalmanFilter(4, 2)  # 4 state variables (x, y, dx, dy), 2 measurement variables (x, y)
kalman.transitionMatrix = np.array([[1, 0, 1, 0],
                                    [0, 1, 0, 1],
                                    [0, 0, 1, 0],
                                    [0, 0, 0, 1]], np.float32)
kalman.measurementMatrix = np.array([[1, 0, 0, 0],
                                     [0, 1, 0, 0]], np.float32)
kalman.processNoiseCov = np.eye(4, dtype=np.float32) * 0.03  # Process noise
kalman.measurementNoiseCov = np.eye(2, dtype=np.float32) * 0.1  # Measurement noise

# Variables for optical flow
lk_params = dict(winSize=(15, 15), maxLevel=2,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Initialize previous frame and tracked points
prev_gray = None
prev_points = None

# Output video writer
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('volleyball_tracking.avi', fourcc, 30.0, 
                      (int(cap.get(3)), int(cap.get(4))))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to grayscale and apply background subtraction
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.normalize(gray, None, 0, 255, cv2.NORM_MINMAX)
    fg_mask = bg_subtractor.apply(gray)
    cv2.imshow("bg_subtractor", fg_mask)
    # fg_mask = cv2.medianBlur(fg_mask, 5)  # Reduce noise
    # cv2.imshow("median_blur", fg_mask)
    # kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (10, 10))
    # fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_CLOSE, kernel)
    # cv2.imshow("fg_mask_closed", fg_mask)
    # # _, fg_mask = cv2.threshold(fg_mask, 200, 255, cv2.THRESH_BINARY)
    # # fg_mask = np.where(fg_mask > 0, 255, 0).astype(np.uint8)  # Convert mask to binary (1 for pixel value > 0, 0 otherwise)
    # cv2.imshow("fg_mask_threshold", fg_mask)
    
    
    # # Detect contours
    # contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # for contour in contours:
    #     area = cv2.contourArea(contour)
    #     if area > 50:  # Only consider contours with significant area
    #         cv2.drawContours(frame, [contour], -1, (255, 0, 0), 2)  # Draw contour in blue

    # detected_ball = None
    # for contour in contours:
    #     area = cv2.contourArea(contour)
        
    #     if 50 < area < 500:  # Filter by area to remove noise
    #         (x, y), radius = cv2.minEnclosingCircle(contour)
    #         if ball_radius_range[0] < radius < ball_radius_range[1]:  # Ensure realistic size
    #             detected_ball = (int(x), int(y))
    #             break  # Prioritize first valid detection

    # if detected_ball:
    #     x, y = detected_ball
    #     measurement = np.array([[np.float32(x)], [np.float32(y)]])
    #     kalman.correct(measurement)
    #     prediction = kalman.predict()
    #     predicted_pos = (int(prediction[0]), int(prediction[1]))

    #     # Draw detected ball and predicted position
    #     cv2.circle(frame, (x, y), ball_radius_range[1], (0, 255, 0), 2)  # Green for detected
    #     cv2.circle(frame, predicted_pos, ball_radius_range[1], (0, 0, 255), 2)  # Red for Kalman-predicted

    

    # Detect circles using Hough Transform
    circles = cv2.HoughCircles(fg_mask, cv2.HOUGH_GRADIENT, dp=1.2, minDist=10,
                               param1=10, param2=20, minRadius=ball_radius_range[0], maxRadius=ball_radius_range[1])
    
    if circles is not None:
        circles = np.uint16(np.around(circles))[0, :]
        ball_center = (circles[0][0], circles[0][1])

        # Kalman filter update
        measurement = np.array([[np.float32(ball_center[0])], [np.float32(ball_center[1])]])
        kalman.correct(measurement)
        prediction = kalman.predict()
        predicted_pos = (int(prediction[0]), int(prediction[1]))

        # Draw detected ball and predicted position
        cv2.circle(frame, ball_center, ball_radius_range[1], (0, 255, 0), 2)
        cv2.circle(frame, predicted_pos, ball_radius_range[1], (0, 0, 255), 2)

        prev_points = np.array([[ball_center]], dtype=np.float32)

    # Optical Flow Tracking
    if prev_gray is not None and prev_points is not None:
        next_points, status, _ = cv2.calcOpticalFlowPyrLK(prev_gray, gray, prev_points, None, **lk_params)
        if status[0][0] == 1:  # Valid tracking
            ball_center = (int(next_points[0][0][0]), int(next_points[0][0][1]))
            cv2.circle(frame, ball_center, ball_radius_range[1], (255, 0, 0), 2)
            prev_points = next_points

    prev_gray = gray.copy()

    # Write frame to output video
    out.write(frame)

    # Display the frame
    cv2.imshow("Ball Tracking", frame)
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

# Cleanup
cap.release()
out.release()
cv2.destroyAllWindows()


C:\Users\harjo\AppData\Local\Temp\ipykernel_6756\3854904383.py:105: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predicted_pos = (int(prediction[0]), int(prediction[1]))


C:\Users\harjo\AppData\Local\Temp\ipykernel_6756\70528062.py:80: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predicted_pos = (int(prediction[0]), int(prediction[1]))


In [3]:
import cv2
import numpy as np

# Load the video
video_path = "volleyball_video.mp4"
cap = cv2.VideoCapture(video_path)

# Check if video loaded successfully
if not cap.isOpened():
    raise ValueError("Error opening video file.")

# Parameters for ball tracking
ball_radius_range = (3, 7)  # Approximate pixel size of the volleyball
bg_subtractor = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=30, detectShadows=True)

# Kalman Filter initialization
kalman = cv2.KalmanFilter(4, 2)  # (x, y, dx, dy) -> 2D position tracking
kalman.transitionMatrix = np.array([[1, 0, 1, 0],
                                    [0, 1, 0, 1],
                                    [0, 0, 1, 0],
                                    [0, 0, 0, 1]], np.float32)
kalman.measurementMatrix = np.array([[1, 0, 0, 0],
                                     [0, 1, 0, 0]], np.float32)
kalman.processNoiseCov = np.eye(4, dtype=np.float32) * 0.03  # Process noise
kalman.measurementNoiseCov = np.eye(2, dtype=np.float32) * 0.1  # Measurement noise

# Variables for tracking
prev_gray = None
ball_position_history = []

# Output video writer
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('volleyball_tracking.avi', fourcc, 30.0, 
                      (int(cap.get(3)), int(cap.get(4))))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to grayscale and apply background subtraction
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cv2.imshow('Gray Frame', gray)  # Plot grayscale frame

    fg_mask = bg_subtractor.apply(gray)
    cv2.imshow('Foreground Mask', fg_mask)  # Plot foreground mask

    fg_mask = cv2.medianBlur(fg_mask, 5)  # Reduce noise
    cv2.imshow('Blurred Foreground Mask', fg_mask)  # Plot blurred mask

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_CLOSE, kernel)
    cv2.imshow('Morphological Closing', fg_mask)  # Plot morphological closing result

    cv2.waitKey(1)  # Allow the window to update
    
    # Enforce brightness threshold (ball is always white)
    _, fg_mask = cv2.threshold(fg_mask, 200, 255, cv2.THRESH_BINARY)

    # Find contours in the fg_mask
    contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    detected_ball = None
    if contours:
        # Find the largest contour by area (likely the ball)
        largest_contour = max(contours, key=cv2.contourArea)
        
        # Apply Convex Hull to merge fragmented parts
        hull = cv2.convexHull(largest_contour)
        
        # Ensure the contour is large enough to be the ball
        if cv2.contourArea(hull) > 50:  # Adjust threshold based on actual video resolution
            # Fit an ellipse (better than HoughCircles for non-perfect circles)
            if len(hull) >= 5:  # Minimum 5 points needed for ellipse fitting
                ellipse = cv2.fitEllipse(hull)
                (x, y), (major_axis, minor_axis), angle = ellipse
                
                # Only consider nearly circular ellipses
                if 0.8 < minor_axis / major_axis < 1.2:
                    detected_ball = (int(x), int(y))

                    # Draw the detected ball
                    cv2.ellipse(frame, ellipse, (0, 255, 0), 2)  # Green ellipse

                    # Kalman filter update
                    measurement = np.array([[np.float32(x)], [np.float32(y)]])
                    kalman.correct(measurement)
                    prediction = kalman.predict()
                    predicted_pos = (int(prediction[0]), int(prediction[1]))

                    # Draw Kalman-predicted position
                    cv2.circle(frame, predicted_pos, 5, (0, 0, 255), 2)  # Red predicted position

                    # Save position history
                    ball_position_history.append(detected_ball)
    
    # Enforce motion: if the ball is too static, remove it
    if len(ball_position_history) > 10:
        last_pos = np.array(ball_position_history[-10])
        current_pos = np.array(ball_position_history[-1])
        if np.linalg.norm(current_pos - last_pos) < 5:
            ball_position_history.pop(-1)  # Remove static detections
    
    # Write frame to output video
    out.write(frame)

    # Display the frame
    cv2.imshow("Ball Tracking", frame)
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

# Cleanup
cap.release()
out.release()
cv2.destroyAllWindows()


C:\Users\harjo\AppData\Local\Temp\ipykernel_6756\4134297168.py:89: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predicted_pos = (int(prediction[0]), int(prediction[1]))
